In [154]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
# Load environment variables from a .env file
load_dotenv()

True

In [121]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [122]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL')
)
model_name = os.getenv("MODEL_NAME")

Get LLM Response

In [123]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome (Italian: Roma).


Prompt engineering 
structured output

In [124]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of 
    "capital": the capital of the country stated
}
]
"""
messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":"What's the capital of Italy?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [125]:
type(response)

str

In [126]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

input structuring


In [127]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    }
]


In [128]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

Give the model time to think (Chain of thought)
https://arxiv.org/pdf/2205.11916


In [ ]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [130]:
259/2*8654+91072*33-12971

4113098.0

In [131]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1343519
}


In [132]:
4113098.0 - 1431449.0


2681649.0

In [133]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [134]:
print(response)

{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 4,115,477.5. Finally, subtract 12971 from 4,115,477.5: 4,115,477.5 - 12971 = 4,005,806.5.",
    "result": 4005806.5
}


In [136]:
4113098.0 - 4005806.5

107291.5

RAG - Retrieval Augmented Generation

Asking about a subject that the LLM does not know anything about

In [137]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

There is no such thing as an "iPhone 16". Apple has not released an iPhone with that model number. The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Mini, iPhone 13 Pro, and iPhone 13 Pro Max.

However, if you're looking for information on the latest rumors and leaks about the next-generation iPhone, I can provide you with some information.

Rumors suggest that the next-generation iPhone, likely to be called iPhone 14, will feature several upgrades and improvements, including:

1. Improved cameras: The iPhone 14 is expected to feature a new triple-camera setup with a wider-angle lens, a telephoto lens, and a ultra-wide-angle lens.
2. Faster processor: The iPhone 14 is expected to be powered by Apple's A15 Bionic chip, which will provide a significant boost in performance and efficiency.
3. Longer battery life: The iPhone 14 is expected to have a longer battery life, with some reports suggesting up to 12 hours of internet use on a

Giving Context to the unknown subject

In [138]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [139]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max.
2. Thinner bezels and a more durable Ceramic Shield.
3. New A18 chip (A18 Pro for the Pro models) with improved performance, neural engine capabilities, and faster GPU for gaming and machine learning tasks.
4. Upgraded camera systems:
	* Base iPhone 16: dual-camera setup with a 48MP main sensor.
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.
5. Advanced audio features, including "Audio Mix" for refined audio capture during video recording.
6. Extended battery life, especially in the iPhone 16 Pro Max, which has the longest-lasting battery of any iPhone.
7. Switch to USB-C for faster charging and data transfer.
8. Pro models support up to 2x faster video encoding.
9. Sta

Automatically extract context data from database

In [140]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [141]:
data = [iphone_16,samsung_s23]

In [142]:
user_prompt = """What's new in iphone 16?"""

In [155]:
embedding_client = OpenAI(
        api_key=os.getenv("RUNPOD_TOKEN"), 
        base_url=os.getenv("RUNPOD_EMBEDDING_URL")
    )

In [ ]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(input=text_input, model=model_name)
    
    embeddings = []  # Corrected the variable name
    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)

    return embeddings
